## Data Ingestion stage

This notebook handles the **data ingestion stage** of the project. Its main purpose is to:
- Download the raw dataset (cats and dogs images) from a specified URL.
- Unzip and extract the images.
- Clean the dataset by removing corrupted or empty files.
- Prepare the data directory for further processing and model training.

In [1]:
# Import required modules for data handling
from collections import namedtuple
import os

In [2]:
# (Optional) Change working directory up one level if needed
os.chdir("../")

In [3]:
# Print the current working directory to confirm location
%pwd

'/home/zkhechadoorian/CNNs_Cats_and_Dogs'

In [4]:
# Define a namedtuple for data ingestion configuration
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

# Define a dataclass for data ingestion configuration (alternative to namedtuple)
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
# Import utility functions and constants for configuration management
from ImageClassification.constants import *
from ImageClassification.utils import read_yaml, create_directories

In [6]:
# Define a configuration manager to read config files and create directories
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        """
        Initialize the ConfigurationManager by reading configuration and parameter YAML files,
        and creating the root artifact directory.
        Args:
            config_filepath (Path): Path to the configuration YAML file.
            params_filepath (Path): Path to the parameters YAML file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Extract the data_ingestion section from the configuration file,
        create the root directory for data ingestion artifacts,
        and return a DataIngestionConfig object.
        Returns:
            DataIngestionConfig: Configuration for data ingestion.
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
# Import additional modules for downloading and extracting files
import os
import urllib.request as request
from zipfile import ZipFile

In [8]:
# Define the DataIngestion class for downloading and cleaning the dataset
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Initialize the DataIngestion class with the given configuration.
        Args:
            config (DataIngestionConfig): Configuration for data ingestion.
        """
        self.config = config

    def download_file(self):
        """
        Download the dataset from the source URL if it does not already exist locally.
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):
        """
        Filter the list of files to include only valid image files (cats and dogs).
        Args:
            list_of_files (list): List of file names from the zip archive.
        Returns:
            list: Filtered list of valid image file names.
        """
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        """
        Extract and clean an image file from the zip archive.
        Removes the file if it is empty or corrupted.
        Args:
            zf (ZipFile): ZipFile object for the dataset archive.
            f (str): File name to extract and preprocess.
            working_dir (str): Directory to extract the file to.
        """
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        """
        Unzip the dataset and clean corrupted or empty files.
        Only valid image files are extracted and cleaned.
        """
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [9]:
# Print the current working directory again (for debugging)
%pwd

'/home/zkhechadoorian/CNNs_Cats_and_Dogs'

In [10]:
# Run the data ingestion pipeline: download and extract/clean the dataset
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2025-09-21 03:41:08,556: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-21 03:41:08,558: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-21 03:41:08,559: INFO: common: created directory at: artifacts]
[2025-09-21 03:41:08,559: INFO: common: created directory at: artifacts/data_ingestion]
